# Notebook to understand Zachary Stangbye's GPR Code

The basic idea is to copy his code line-by-line, and annotate it with paragraphs/comments that help me understand it as I go along.

My understanding is that the basic order of the code is the following:

1. Parameter definitions
2. Machine learning setup (grid-points generation using typicality criteria, hyperparameter bounds for GPR, state space transformation, covariance function)
3. GPR implementation (log-likelihood function for optimization, functions to create the covariance matrix, GPR approximation functions, and storage for coefficients and transformed values
4. Equilibrium object definitions
5. Model-specific objects (functions to compute new bond prices, value functions for repayment and default, expected future values with quadrature integration
6. VFI - main solution loop. Updates pricing function, value functions and policy functions, optimizes GPR hyper-parameters at each iteration, and computes coefficient vectors for GPR approximations

A significant portion of this code is dedicated to bounding, then logit-transforming and finally normalizing the desired objects. All of this is for the sake of improving performance and stability within the GPR process.

Here's a quick detour/aside on logit transformations: If you look it up on the internet, you'll find the following formula: 

**logit(p) = ln(p/(1-p))**, where p is between 0 and 1. 

When our object is between [a,b] instead of [0,1], we transform it with:

**normalized_x = (x-a)/(b-a)**

With a bit of rearranging, you will see the forms that Zach uses in his code!

# 1.Declaring economic parameters

In [1]:
const max_iters = 600
const r = 0.017
const β = 0.953
const σ_CRRA = 2.0

2.0

In [2]:
function sov_utility(c)
    u = 1.0
    if c <= 0.0
        u = -1.0e10
    else
        u = c^(1-σ_CRRA)/(1-σ_CRRA)
    end
    return u
end

sov_utility (generic function with 1 method)

In [3]:
const σy = 0.025 # Output shock volatility - it's like the η parameter in the QuantEcon version of this code
const ρy = 0.945 # Output persistence      - it's like the ρ parameter in the QuantEcon version of this code
const σy_uncond = σy/sqrt(1-ρy^2)

0.07643616004405837

### 1.1 Approximately generates the threshold-cost specification of Arellano (2008)

In [4]:
const y_const = -0.5 
const y_curv = 0.53 
ydef(y) = y - max(0.0,y_const*y + y_curv*y^2)

ydef (generic function with 1 method)

In [5]:
const π_re_enter = 0.282 # Re-entry probability - it's the θ parameter in the Quantecon version of this code

0.282

# 2. Declaring numerical parameters

## 2.1 Bounds on y, b, q, V

In [6]:
const stds_out = 3.0

const yL = exp(-stds_out*σy_uncond)  #lowest point on the endowment grid
const yH = exp(stds_out*σy_uncond)   #highest point on the endowment grid

const bLopt = 0
const bHopt = 0.3 

const bL = bLopt-1.0e-6     #Lowest permitted borrowing
const bH = bHopt+1.0e-6     #Highest permitted borrowing

const qL = 0.0              #Lowest possible price (most expensive)
const qH = 1/(1+r)          #Highest possible price (least expensive - the risk free price)

const VL = -50.0 #-100.0
const VH = -1.0e-6

const bN_opt = 50
const b_opt_grid = bLopt:(bHopt-bLopt)/(bN_opt-1):bHopt

0.0:0.006122448979591836:0.3

In [7]:
# ML parameters
const D = 2 #dimensionality of the problem - y and b in this case
const N_inputs = 30*D #30 points per dimension in this case...
# Scheidigger and Bilionis (2019) suggest 5*D or 10*D for N_inputs, but we
# find this to be insufficient given the significant non-linearities in these models
# However, the golden feature that the problem is linear in D remains
const N_validate = 10_000   #off-grid training points to validate that it's all correct...

10000

# 3. Loading packages

In [8]:
using FastGaussQuadrature, Random, StatsFuns
using LinearAlgebra, Optim
using LaTeXStrings, Plots
gr() # Sets the backend for the plotting interface (not used in solution)

Plots.GRBackend()

# 4 Parameterizing expectations

In [9]:
# Take expectations with a Gauss-Chebyshev quadrature
# Given that there are significantly fewer grid points, it's important
# for this to be reasonably large for precise expectations
const quad_size = 15
const gc_points, gc_weights = gausschebyshev(quad_size)

#Quadrature nodes and weights, I guess....

# Minor adjustment to quadrature-based expectations to ensure pdf integrates to unity
const pdf_adj = normcdf(stds_out)-normcdf(-stds_out)
#All of this adds up to pdf_adc...which in this case, is 0.997. Rescaling it so it integrates to 1.0

0.9973002039367398

# NOT REALLY SURE WHAT THIS IS YET - NEED TO UPDATE

In [11]:
eyeN1 = zeros(N_inputs,N_inputs)

#N-inputs is the number of gridpoints. Since there are 2 dimensions and 30 points per dimension, it's 60!
#N-dimensional identity matrix....why though?
for i=1:N_inputs
    eyeN1[i,i] = 1.0
end

const eyeN = copy(eyeN1)

60×60 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

# 5. Drawing inputs/gridpoints (fixed for all iterations) from typical sets

## 5.1. Why are we doing this/why aren't we just using a uniformly spaced grid?

I don't think this part is strictly needed, but it helps with the code's efficiency.

Basically, the Arellano (2008) model has an IRREGULAR state space. I.e. the economically relevant region of the state space doesn't fill up the entire hypercube defined by the upper and lower bounds of the state variables. Instead, the ergodic distribution of states (wthe states the economy naturally visits during simulations) forms a more complex shape like a hypersphere, simplex, etc. 

Putting aside how this irregular state space is constructed, creating it is important because if you create a normal uniformly-spaced grid, you'd be evaluating the function where it doesn't matter - it'd be very inefficient. You want to allocate your computational resources where the economy is actually highly likely to spend alot of time. I guess this importance becomes even bigger in higher dimensional spaces.

## 5.2. So how does Zach draw out the training inputs? I.e. what is this code doing below?

### 5.2.1. Gridpoint setup

In [12]:
gridPointsTemp = zeros(2,N_inputs)
gridPointsTempVec = zeros(1,N_inputs)

1×60 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

**gridPointsTemp** is a 2 by N_inputs (2 by 30 in this case) matrix. It stores the coordinates for points in Arellano's 2D state-space.
1. Row 1 (index 1) stores value for output (y)
2. Row 2 (index 2) stores values for debt (b)

**gridPointsTemp[:,i]** makes it easy to access an entire state point

**gridPointsTemp[1,:]** makes it easy to access all output values (y)

**gridPointsTemp[2,:]** makes it easy to access all debt values (b)

**gridPointsTempVec** seems to be a temporary 1 by N_inputs vector that is used as an intermediate step when generating gridpoints for y. However, I don't think it actually does anything...

### 5.2.2 How points for debt (b) are drawn

**Stratified sampling for DEBT**

**b_thresh** (0.225 in this case) is the threshold at which debt values below it are considered low, and debt values above it are considered high. 

**N_thresh** (20 in this case) is the number of gridpoints allocated to the low debt region. The remaining (40) points are allocated to the high debt region.

The actual point creation might look a bit complicated, but **(i-1.0)/(N_thresh-1.0)** is just a value that goes from 0 to 1 as i goes from 1 to N_thresh, and all of this just ensures that you are creating a sequence of equally spaced points.

In [13]:
const b_thresh = .75*bHopt-bLopt 
const N_thresh = 10*D

# We draw debt gridpoints (endogenous) from a uniform grid, adding a few more points 
# in high-debt areas even though the sovereign never goes there
for i=1:N_inputs
    gridPointsTempVec[i] = yL + (yH - yL)*(i-1.0)/(N_inputs-1.0)
end 
for i=1:N_thresh
    gridPointsTemp[2,i] = bLopt + (b_thresh-bLopt)*(i-1.0)/(N_thresh-1.0)
end 
for i=N_thresh+1:N_inputs
    gridPointsTemp[2,i] = b_thresh + (bHopt-b_thresh)*(i-N_thresh-1)/(N_inputs-N_thresh-1)
end

### 5.2.3. How training points for output are drawn

**Typical set sampling for OUTPUT**

Zach does something fancy from information theory. Basically, if you have a random variable, you want to obtain its **typical set**, which contains outcomes that aren't extremely unlikely or likely. It's basically a computationally efficient way of sampling from the distribution.

At a high level, this code draws N random samples from a normal distribution, calculates the empirical entropy of the entire sample, and accepts the entire batch of N points ONLY if the empirical entropy is close to the theoretical entropy. If it's not, then the code will keep redrawing new samples until it's close enough.

1. The code draws random normal samples: **z_draw = σy_uncond*randn()**
2. Converts it to log normal: **gridPointsTemp[1,i] = exp(z_draw)**
3. Calculates the entropy of this sample: **z_draw_entropy = z_draw_entropy - log(normpdf(z_draw/σy_uncond)/σy_uncond)/N_inputs**
4. Accepts the sample only if the empirical entropy is close enough to the theoretical entropy: **abs(z_draw_entropy - z_entropy) < typ_thresh**

In [14]:
Random.seed!(13) # Set random seed so get same answer each time
const typ_thresh = 1.0e-6 
#  y is drawn to be unconditionally typical
const z_entropy =  .5*log(2.0*π*ℯ*σy_uncond^2.0)
not_typical = true
while (not_typical)
    z_draw_entropy = 0.0
    for i=1:N_inputs 

        z_draw = σy_uncond*randn()
        gridPointsTemp[1,i] = exp( z_draw )
        z_draw_entropy = z_draw_entropy - log(normpdf(z_draw/σy_uncond)/σy_uncond)/N_inputs

    end

    if (  abs(z_draw_entropy - z_entropy) < typ_thresh ) 

        global not_typical = false 

    end 
end

## 5.3 What actually goes into the GPR?

So the reason it's called **gridPointsTemp** is because these aren't actually the gridpoints that we feed into the GPR. The values within here have their normal economic interpretation. However, for optimization/performance reasons, GPRs like it when the inputs are all normalized to be between 0 and 1.

I.e. we actually feed a rescaled version of **gridPointTemp**, called **scaledGridPoints** into the GPR. The rescaling is done by dividing each row by the lower bound [yL;bL], and dividing it by the range [yH - yL, bH - bL).

In [15]:
const gridPoints = copy(gridPointsTemp)
const scaledGridPoints = (gridPointsTemp .- [yL;bL] )./([yH - yL; bH - bL])

2×60 Matrix{Float64}:
 0.563737    0.427194   0.40771    0.530702  …  0.311511  0.593914  0.269368
 3.33331e-6  0.0394768  0.0789502  0.118424     0.987176  0.993586  0.999997

## 5.4. Solution validation

The **drawConvergenceInputs** function draws random points from the state space to check for convergence. I.e. it doesn't check for convergence, but it generates num_draws number of points in the state space to validate the solution.

In [16]:
function drawConvergenceInputs(num_draws)
    xx = zeros(2,num_draws)
    for i=1:num_draws 
        xx[:,i] = rand(2,1).*[yH-yL;bH-bL] .+ [yL;bL]
    end
    return xx
end

drawConvergenceInputs (generic function with 1 method)

# 6. Initializing the GPR

## 6.1 Covariance function
Zach uses the standard square exponential covariance function, but no **automatic-relevance-determination** (I think this means that the l's are all set to be the same, rather there being a different l for every training input).

It takes the following inputs:
1. x is simply the vector of training inputs
2. xprime is the other other vector of training inputs - NEED TO VERIFY
3. thetavec is the the vector of hyperparameters - 2D in this case. The first element is the **signal strength**, and the second one is the **length scale**.

It's not that intimidating - it's simply on Zach's slides for Gaussian Process Regression IV.

In [17]:
function k_SE(x,xprime,θvec)
    s = θvec[1]
    l = θvec[2]
    return exp( -0.5*sum( ( (x .- xprime)/l ).^2 ) + 2*log(s) )
end

k_SE (generic function with 1 method)

## 6.2 Covariance matrix

This function creates the **covariance matrix**, which is used both in likelihood maximization (**negLL function**) and in the derivation of the GPR coefficients later on in the line: VR_gprCoeff = 

It basically applies the k_SE function we just defined to every training input and returns a N by N covariance matrix.

There is a clever trick though - note the second conditional within the two for-loops and then how the index values are flipped. What's happening, is that the for-loops are first only calculating the covariances of the **upper-half** of the covariance matrix. Then the second if statement just mirrors it, because we know that covariance matrices are symmetrical. 

I.e. the computation time gets halved!

In [18]:
function createK(hyper_params,num_dims)

    currK = zeros(N_inputs,N_inputs)

    for ik1=1:N_inputs 
        for ik2=1:ik1 

            if (num_dims == 1) 
                currK[ik1,ik2] = k_SE(scaledGridPoints[1,ik1],scaledGridPoints[1,ik2],hyper_params)
            else 
                currK[ik1,ik2] = k_SE(scaledGridPoints[:,ik1],scaledGridPoints[:,ik2],hyper_params)
            end 

            if (ik1 != ik2) 
                currK[ik2,ik1] = currK[ik1,ik2]
            end 

        end 
    end 

    return currK

end  

createK (generic function with 1 method)

## 6.3 Log-likelihood

### 6.3.1 Bounds on the hyper-parameters

Since this the likelihood is really important, it's crucial that it is numerically stable. Zach thus places bounds on the hyperparameters to prevent the function from exploring parameter values that are way too high. 

**kernSSLogLB** and **kernSSLogUB** define the bounds for the log of the **signal strength hyperparemeter**. (The first hyperparameter of the kernel)

**kernLogLB** and **kernLogUB** define the bounds for the log of the **length scale hyperparameter**. (The second hyperparameter of the kernel)

You might also be wondering why -2 and 8. That's because these hyperpameters are optimized in log-space. (10^log_hyper_params). I.e. the actual values explored here are between 10^-2 and 10^8.

In [19]:
const kernSSLogLB = -2.0
const kernSSLogUB = 8.0
const kernLogLB = -2.0
const kernLogUB = 8.0

8.0

**init_kernCoeff** gives the initial guess for the hyper-parameters when the algorithm (Nelder-Meade in Zach's case) looks for the optimal set of hyperparameters

In [20]:
const init_kernCoeff = [1.0;1.0] # Ensure this guess gives finite log-likelihood

const sn_star = 1.0e-4 # Assumed 'measurement' noise -> sn_star^2 = 1.0e-8

0.0001

### 6.3.2. Log-likelihood function

The **negLL** function computes the negative of the log-likelihood for the GPR (it's negative since the Optim package finds the minimum of a function, rather than the maximum).

The inputs are the following:
1. **log_hyper_params**: logarithms of the hyperparameters (signal strength and length scale)
2. **t_set**: training output values
3. **num_dims**: dimensionality of the input space (this matters because the value function is only 1-D, whereas it's 2D for everything else)

It works in the following order:
1. Check if the log-parameters are within the bounds set above
2. Convert the hyperparameters from log-space to regular-space
3. Compute the covariance by calling the createK function
4. Add measurement noise to the diagonal
5. Calculate the log-likelihood.

The code also does some error handling. If the parameters are out of bounds, or the covariance matrix is not positive definite (i.e. negative determinant), then it'll return a large value to steer the optimizer away.

In [21]:
function negLL(log_hyper_params, t_set, num_dims) 

    negLL1 = 0.0

    if ( (log_hyper_params[1] > kernSSLogLB) && (log_hyper_params[1] < kernSSLogUB) && (log_hyper_params[2] > kernLogLB) && (log_hyper_params[2] < kernLogUB) )

        hyper_params = 10.0.^log_hyper_params #Convert log-space hyperparams to regular space

        currK =  createK(hyper_params,num_dims) #Calculate the covariance matrix

        Ssigma = currK + sn_star^2.0*eyeN  #Add the noise term to the diagonal

        # This way uses the standard log(|Sigma|) approach
        temp_num = det(Ssigma)

        if (temp_num > 0.0) 
            negLL1 = log(temp_num) + sum(t_set'/Ssigma*t_set)
        else 
            negLL1 = 1.2e10
        end  

    else # If it violates the bounds, return a big number
        negLL1 = 1.0e10 

    end  

    return negLL1

end 

negLL (generic function with 1 method)

## 6.5 GPR_approx function

This function implements the GP posterior mean function, which predicts the function value at any point in the state space based on the training data. 

It takes in the following inputs:
1. x: a SINGLE point where I want to evaluate the GP. In this problem, it's a single 2D vector (b,y) where I want to predict the function value.
2. AA: GPR coefficients like VR_gprCoeff computed earlier by solving the linear system.
3. kern_params: Hyperparameters of the kernel function (singal strength and length scale)
4. xx: Matrix of of ALL training input points. I.e. scaledGridPoints

It's simply implementing the tilde{m(x)} function on Slide 18, or GPR posterior. Somewhat confusingly, Zach flips the order around. AA is (K + \sigma^2 etc.) that's been pre-computed! Note that GPR_approx1 is set to 0.0 because we assume a zero-mean.

In [22]:
function GPR_approx(x,  #Training input
                    AA, #GPR coefficients
                    kern_params, #Kernel hyperparameters (signal strength and length scale)
                    xx)  #Entire matrix of training inputs (scaledGridPoints?)     

    GPR_approx1 = 0.0

    for i=1:N_inputs
        GPR_approx1 = GPR_approx1 + AA[i]*k_SE(x,xx[:,i],kern_params)
    end 

    return GPR_approx1
end   

GPR_approx (generic function with 1 method)

## 6.6 Constants

This part of the code transforms the economic variables into oones that are suitable for GPR approximation. The functions we care about are usually bounded, but the algorithm can in principle produce any real number. Hence, we transform it to remove this possibility.

There's alot going on, so here's how to read all this information: 
* If the variable has a z suffix, then they are logit-transformed. They transform bounded values to an unbounded range.
* If the variable doesn't have a z suffix, then it is normalized transformed. I.,e., it;'s what actually gets fed into the GPR. They are centralized (zero mean) and standardized (unit variance).

Basically, in the majority of the VFI step, we're dealing with variables of economic significance (e.g. vnew). Then at the GPR stage, to avoid the aforementioned issus, it gets **logit-transformed** (e.g. tset_VRz), then **normalized** (e.g. tset_VR). We need all this overhead to transform these normalized values back into values of economic significance.

In [23]:
# These vectors will hold the original (logit-transformed) output values
tset_VDz = zeros(N_inputs)
tset_VRz = zeros(N_inputs)
tset_qz = zeros(N_inputs)
tset_Az = zeros(N_inputs)

# These vectors will hold the cleaned (logit-transformed) output values
tset_VD = zeros(N_inputs)
tset_VR = zeros(N_inputs)
tset_q = zeros(N_inputs)
tset_A = zeros(N_inputs)

60-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

### 6.6.1. Remarks on the initial constants

There is nothing inherently special about 500 and -500 specifically, we just needed it to be a very big number. If it's positive (500), then this corresponds to an initial guess of it being towards the lower bound. If it's negative, then it's close to the upper-bound. Note that this isn't declared as a constant, and indeed, it's it changes with every single iteration.

The same happens for all the standard deviations. They are just default starting values that will get updated with each transformation.

In [24]:
# These scalars are used to clean the output values 
mean_VD = 500.0 # With the logit transform, this corresponds to an initial zero
mean_VR = 500.0 # With the logit transform, this corresponds to an initial zero
mean_q = -500.0 # With the logit transform, this corresponds to an initial zero
mean_A = -500.0 # With the logit transform, this corresponds to an initial zero

std_VD = 1.0 
std_VR = 1.0 
std_q = 1.0 
std_A = 1.0

# These store the kernel coefficients for the approximations (signal strength, scalelength)
VR_kernCoeff = ones(2)
VD_kernCoeff = ones(2)
q_kernCoeff = ones(2)
A_kernCoeff = ones(2)

# These store the GPR coefficients for the approximation 
VR_gprCoeff = zeros(N_inputs)
VD_gprCoeff = zeros(N_inputs)
q_gprCoeff = zeros(N_inputs)
A_gprCoeff = zeros(N_inputs)

60-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

# 7. Equilibrium objects

## 7.1. Value function for repayment

These are the four functions that we want the GPR to approximate. They calculate the value of repaying, defaulting, etc. for a given **xIn**. The main thing that's a bit confusing is what this function returns.
1. Normalize all the inputs (which are already logit transformed). into a [0,1] range
2. Apply the GPR to get a prediction in the transformed space
3. Apply the inverse logit transformation to map this value back into the original value range (e.g. VL, VH)

The logit transformation earlier was:
tset_VRz[i] = -log((VH-VL)/(vnew-vL) - 1.0)

The majority of this line is just computing the inverse of it - i.e. undoing that logit transformation.

**GPR_approx(xTransformed,VR_gprCoeff,VR_kernCoeff,scaledGridPoints)** is step 2 - applying the GPR.

Deducting **mean_VR** and multiplying by **std_VR** is **un-normalizing** the value. The rest is undoing the **logit transformation**. The output is now of economic significance!

However, these functions don't contain economic structure within them! They just house GPR evaluation, and are purely computational. The actual economic logic is housed within the functions in the next section with the **new_** prefix.

In [25]:
function vRepay(xIn) # Repayment value (Y_t, B_t)

    xTransformed = zeros(2) 
    xTransformed[1] = (xIn[1] - yL)/(yH - yL) #Normalizes y to between [0,1]
    xTransformed[2] = (xIn[2] - bL)/(bH - bL) #Normalizes b to between [0,1]
    #This normalization apparently helps the GPR work better by keeping all inputs in a constant range
    return VL + (VH - VL)/(1.0 + exp(-std_VR*GPR_approx(xTransformed,VR_gprCoeff,VR_kernCoeff,scaledGridPoints)-mean_VR))

end

vRepay (generic function with 1 method)

## 7.2. Value function for default

In [26]:
function vDefault(xIn) # Default value (Y_t)

    xTransformed = (xIn - yL)/(yH - yL)

    return VL + (VH - VL)/(1.0 + exp(-std_VD*GPR_approx(xTransformed,VD_gprCoeff,VD_kernCoeff,scaledGridPoints[1,:]')-mean_VD))

end

vDefault (generic function with 1 method)

## 7.3. Bond pricing function

In [27]:
function q(xIn) # Pricing function (Y_t, B_t+1)

    xTransformed = zeros(2) 
    xTransformed[1] = (xIn[1] - yL)/(yH - yL)
    xTransformed[2] = (xIn[2] - bL)/(bH - bL)

    return qL + (qH - qL)/(1.0 + exp(-std_q*GPR_approx(xTransformed,q_gprCoeff,q_kernCoeff,scaledGridPoints)-mean_q))

end

q (generic function with 1 method)

## 7.4: Bond issuance policy function

In [28]:
function Apol(xIn) # Borrowing policy (Y_t, B_t)

    xTransformed = zeros(2) 
    xTransformed[1] = (xIn[1] - yL)/(yH - yL)
    xTransformed[2] = (xIn[2] - bL)/(bH - bL)

    return bL + (bH - bL)/(1.0 + exp(-std_A*GPR_approx(xTransformed,A_gprCoeff,A_kernCoeff,scaledGridPoints)-mean_A))

end 

Apol (generic function with 1 method)

## 7.5 Default policy function

**defFun** is the default policy function. It takes in the state variables (just x for (b,y)), and returns 1.0 or 0.0 (from the Float64 wrapper) depending on if defaulting is more valuable than repaying. It's used in the new_q function.

I think this is an auxillary function though.


In [29]:
defFun(x) = Float64(vDefault(x[1]) > vRepay(x))

defFun (generic function with 1 method)

## 7.6: Associated log-likelihood functions

For every single function that we want to approximate with a GPR, Zach defines an individual, specicialized function which serve as objective functions for the optimization procedure that finds each hyperparameters for each GPR. 

I.e. later in the code, we'll see lines like **rresultsVR = optimize(negLL_VR, init_kernCoeff, NelderMead())**

Zach separates these functions out because it makes optimization calls cleaner, and crucially, allows each function to have its own set of optimized hyper-parameters, separate from the others!

In [30]:
negLL_VR(log_hyper_params) = negLL(log_hyper_params,tset_VR,2)
negLL_VD(log_hyper_params) = negLL(log_hyper_params,tset_VD,1)
negLL_q(log_hyper_params) = negLL(log_hyper_params,tset_q,2)
negLL_A(log_hyper_params) = negLL(log_hyper_params,tset_A,2)

negLL_A (generic function with 1 method)

# 8. Defining model-specific objects

These functions are the **true** functions calculated directly from the model's economic equations. They compute the actual values based on current approximations. I.e. they are what generate new training data for the GPRs. 

## 8.1 New bond pricing function

This is a function that delivers the pricing function given current sovereign behavior in the next period.

1. **zL1** and **zH1** are lower and upper bounds, which are centered around the expected log output for next period **ρy*log(current_y)**.
2. **z_gc_points =  zL1 .+ ( 1.0 .+ gc_points)./2.0 .*(zH1-zL1)** transforms the Gauss-Chebyshev points from their original **[-1,1]** domain to the new new integration domain **[zL1, zH1]**.
3. For each quadrature point (**1:quad_size**), the loop calculates the weighted probabiltiy contribution of default in that specific future output level. 

**ytempObj** is just a 1.0 or 0.0 - it stores the default decision.
**sqrt(1.0-gc_points[iy]^2.0)** is a correction factor specific to Chebyshev quadrature.
**normpdf((z_gc_points[iy] - ρy*log(current_y))/σy)/σy** is the probability density of this particular realization of future output, according to the AR(1) process.
**/pdf_adj** adjustment factor to account for the trunction of the normal distribution, since we're only integrating over a 3.0 +/- std dev range.

The sum of these points **sum(yvec)**, after being scaled for the integration domain domain with **(zH1-zL1)/2.0**, gives the probability of default.

The final output is the bond price!

In [31]:
function new_q(current_y,issued_b)

    zL1 = ρy*log(current_y) - stds_out*σy
    zH1 = ρy*log(current_y) + stds_out*σy

    z_gc_points =  zL1 .+ ( 1.0 .+ gc_points)./2.0 .*(zH1-zL1)
    yvec = copy(z_gc_points)

    for iy=1:quad_size 

        ytempObj = defFun([exp(z_gc_points[iy]);issued_b])

        yvec[iy] = gc_weights[iy]*ytempObj*sqrt(1.0-gc_points[iy]^2.0)*normpdf( (z_gc_points[iy] - ρy*log(current_y))/σy )/σy/pdf_adj

    end 
    new_def1 = (zH1-zL1)/2.0*sum(yvec)

    return (1.0-new_def1)/(1.0+r)

end

new_q (generic function with 1 method)

## 8.2. New value function for repayment

Everything up to the for-loop is the exact same. Even most of the objects within the for-loop is the same too, it's just that instead of recording the default decisions, it's now recording the continuation value. Immediately outside the for-loop, **CV** is the expected continuation value.

**flow_u** is the flow utility, or just u(c).

In [32]:
function new_VR(current_y,current_b, issued_b)

    zL1 = ρy*log(current_y) - stds_out*σy
    zH1 = ρy*log(current_y) + stds_out*σy

    z_gc_points =  zL1 .+ ( 1.0 .+ gc_points)./2.0 .*(zH1-zL1)
    yvec = copy(z_gc_points)

    for iy=1:quad_size 

        ytempObj = max( vRepay([exp(z_gc_points[iy]);issued_b]), vDefault(exp(z_gc_points[iy]) ) )

        yvec[iy] = gc_weights[iy]*ytempObj*sqrt(1.0-gc_points[iy]^2.0)*normpdf( (z_gc_points[iy] - ρy*log(current_y))/σy )/σy/pdf_adj

    end 
    CV = (zH1-zL1)/2.0*sum(yvec)

    flow_u = sov_utility( current_y - current_b + q([current_y;issued_b])*issued_b )

    return flow_u + β*CV

end

new_VR (generic function with 1 method)

## 8.3: New value function for default

This really is all the same as the value function for repay, only expect that now, we need to account for the probability of re-entering financial markets or not.

In [33]:
function new_VD(current_y)

    zL1 = ρy*log(current_y) - stds_out*σy
    zH1 = ρy*log(current_y) + stds_out*σy

    z_gc_points =  zL1 .+ ( 1.0 .+ gc_points)./2.0 .*(zH1-zL1)
    yvec = copy(z_gc_points)

    for iy=1:quad_size 

        ytempObj = π_re_enter*vRepay([exp(z_gc_points[iy]);0.0]) + (1.0-π_re_enter)*vDefault(exp(z_gc_points[iy]) )

        yvec[iy] = gc_weights[iy]*ytempObj*sqrt(1.0-gc_points[iy]^2.0)*normpdf( (z_gc_points[iy] - ρy*log(current_y))/σy )/σy/pdf_adj

    end 
    CV = (zH1-zL1)/2.0*sum(yvec)

    flow_u = sov_utility( ydef( current_y)  )

    return flow_u + β*CV

end

new_VD (generic function with 1 method)

# 9: Value function iteration

This is value function iteration operating as the limit of a finite-horizon game. I'm not sure what this means yet....

In [34]:
v_old_points = zeros(N_validate)
v_new_points = zeros(N_validate)
const conv_tol = 1.0e-6 
ddist = 1.0
iiters = 0 

0

In [30]:
while ( (ddist > conv_tol) && (iiters < max_iters)) 

    global iiters += 1

    convergence_points = drawConvergenceInputs(N_validate)
    for i=1:N_validate 
        v_old_points[i] = vRepay(convergence_points[:,i])
    end

    #FIRST, we update the pricing equation (from penultimate period on)

    #This is a pretty cool way to incorporate Andres' suggestion about using iteration numbers to change updating rules...
    #Anyway, in the first iteration, (somewhat confusing in the else part, the initial guess of the pricing function is a fixed value.
    #This basically pushes the bond price close to qL (high default probability)
    #In subsequent iterations, the actual price is calculated with new_q(ytod,bissued)
    #Notice that there are the min() and max() functions in there...
    #There are there to bound the actual price to be between qL (the max part) and qH (the min part)
    #There's also the 1.0e-6 - these offsets are just to prevent division by zero, or log(zeros) in the transformation)
    for i=1:N_inputs 
        ytod = gridPoints[1,i]
        bissued = gridPoints[2,i]
        #Note the z suffix - these are logit-transformed!
        if ( iiters > 1) 
            tset_qz[i] = -log((qH-qL)/(min( qH-1.0e-6, max( new_q(ytod,bissued), qL + 1.0e-6)-qL) ) - 1.0)
        else 
            tset_qz[i] = -log((qH-qL)/(1.0e-6-qL) - 1.0)
        end  
    end
    # SECONDLY we `clean' the outputs before running the GPR 
    #1. calculate the mean of all the logit-transformed pricing values
    global mean_q = sum(tset_qz)/N_inputs
    #2. Calculate the std deviatiopn of all these values
    global std_q = sqrt( sum((tset_qz .- mean_q).^2.0)/(N_inputs-1.0) )
    #This is a safety check to avoid division by very small numbers
    if (std_q < 1.0e-6) 
        global std_q = 1.0 
    end
    #3. Create a normalized version (tset_q) by subtracting the mean and dividing it by the standard deviation
    global tset_q = (tset_qz .- mean_q)./std_q

    #4. We now have the updated training outputs for the bond pricing function - we can optimize the hyperparameters now
    #Specifically, we updated one of the key inputs for negLL_q
    #Optimizing the GP likelihood for the bond PRICING FUNCTION
    global rresultsq = optimize(negLL_q,init_kernCoeff,NelderMead())

    global opt_paramsq = rresultsq.minimizer 
    global opt_LLq = -rresultsq.minimum

    global q_kernCoeff = 10.0.^opt_paramsq
    global K_q = createK(q_kernCoeff,2)
    global q_gprCoeff = (K_q .+ sn_star^2.0 .*eyeN)\tset_q

    # LASTLY, we update value and policy functions

    #This is honestly, a pretty cool way to mix both continuous and discrete optimization
     for i=1:N_inputs 
        #Initializing the specific point within the state-space
        ytod = gridPoints[1,i]
        btod = gridPoints[2,i]

        #1.This part implements the optimization on a DISCRETE grid
        #Remember that earlier, Zach initialized a 50-element grid of possible choices for debt issuance 
        vtemp = zeros(bN_opt)
        max_i = 1
        best_v = -1.0e10 #Negative number that is massive in scale
        #Calculating the value in everything within the b_opt_grid
        for iopt=1:bN_opt
            vtemp[iopt] = new_VR(ytod,btod, b_opt_grid[iopt])
            #As my code loops through each possible debt level in the discrete grid, it calculates the value of repayment in that choice
            #As a better value gets calculated, best_v gets updated!
            if (vtemp[iopt] >= best_v) 
                best_v = vtemp[iopt]
                max_i = iopt 
            end 
        end
        #2. This part of the code refines the solution with CONTINUOUS optimization
        #First, upper and lower bounds for the optimization are created
        bL_local = bLopt 
        bH_local = bHopt
        if (max_i == 1) 
            bL_local = bLopt
            bH_local = b_opt_grid[2]
        elseif (max_i == bN_opt)  
            bL_local = b_opt_grid[bN_opt-1]
            bH_local = b_opt_grid[bN_opt]
        else 
            bL_local = b_opt_grid[max_i-1]
            bH_local = b_opt_grid[max_i+1]
        end 
        #Defining the function to be optimized
        v_obj(bprime) = -new_VR(ytod,btod,bprime)
        #Continuous optimization
        rresults = optimize(v_obj,bL_local,bH_local)
        #Storing the value function
        vnew = -rresults.minimum 
        #Storing the policy function
        anew = rresults.minimizer

        #We have the value function and policy function training data in normal form, now let's logit transform it
        tset_VRz[i] = -log((VH-VL)/(vnew-VL) - 1.0)
        tset_Az[i] = -log((bH-bL)/(anew-bL) - 1.0)

        #Updating the training data for the value of defaulting
        vdnew = new_VD(ytod)
        #Then, logit transforming it
        tset_VDz[i] = -log((VH-VL)/(vdnew-VL) - 1.0)

    end 
    
    # Now we `clean' the outputs before running the GPR 
    global mean_VR = sum(tset_VRz)/N_inputs
    global std_VR = sqrt( sum((tset_VRz .- mean_VR).^2.0)/(N_inputs-1.0) )
    if (std_VR < 1.0e-6) 
        global std_VR = 1.0 
    end
    global tset_VR = (tset_VRz .- mean_VR)./std_VR

    global mean_VD = sum(tset_VDz)/N_inputs
    global std_VD = sqrt( sum((tset_VDz .- mean_VD).^2.0)/(N_inputs-1.0) )
    if (std_VD < 1.0e-6) 
        global std_VD = 1.0 
    end
    global tset_VD = (tset_VDz .- mean_VD)./std_VD

    global mean_A = sum(tset_Az)/N_inputs
    global std_A = sqrt( sum((tset_Az .- mean_A).^2.0)/(N_inputs-1.0) )
    if (std_A < 1.0e-6) 
        global std_A = 1.0 
    end
    global tset_A = (tset_Az .- mean_A)./std_A

    # Now, we optimize all the GP likelihoods and get the new functions
    #1. Optimizing the GP likelihood of the value function for REPAYING
    global rresultsVR = optimize(negLL_VR,init_kernCoeff,NelderMead())

    global opt_paramsVR = rresultsVR.minimizer 
    global opt_LLVR = -rresultsVR.minimum

    global VR_kernCoeff = 10.0.^opt_paramsVR
    global K_VR = createK(VR_kernCoeff,2)
    global VR_gprCoeff = (K_VR .+ sn_star^2.0 .*eyeN)\tset_VR

    #2. Optimizing the GP likelihood of the value function for DEFAULTING
    global rresultsVD = optimize(negLL_VD,init_kernCoeff,NelderMead())

    global opt_paramsVD = rresultsVD.minimizer 
    global opt_LLVD = -rresultsVD.minimum

    global VD_kernCoeff = 10.0.^opt_paramsVD
    global K_VD = createK(VD_kernCoeff,1)
    global VD_gprCoeff = (K_VD .+ sn_star^2.0 .*eyeN)\tset_VD

    #3. Optimizing the GP likelihood of the bond issuance POLICY FUNCTION
    global rresultsA = optimize(negLL_A,init_kernCoeff,NelderMead())

    global opt_paramsA = rresultsA.minimizer 
    global opt_LLA = -rresultsA.minimum

    global A_kernCoeff = 10.0.^opt_paramsA
    global K_A = createK(A_kernCoeff,2)
    global A_gprCoeff = (K_A .+ sn_star^2.0 .*eyeN)\tset_A


    # Now update the value function and check for convergence
    for i=1:N_validate 
        v_new_points[i] = vRepay(convergence_points[:,i])
    end
    global ddist = maximum(abs.(v_new_points .- v_old_points))
    println([iiters ddist])
end

[1.0 2.454682693372476]
[2.0 1.1437133080271664]
[3.0 1.0754592658763258]
[4.0 1.0669422852443091]
[5.0 1.012335233029006]
[6.0 0.9425843647329799]
[7.0 0.8762471172979787]
[8.0 0.8358110392593829]
[9.0 0.8058939333831887]
[10.0 0.761173074497755]
[11.0 0.7425961430738539]
[12.0 0.6775979423138239]
[13.0 0.6434304270386093]
[14.0 0.6221677859306283]
[15.0 0.5786808405785209]
[16.0 0.5491957418061091]
[17.0 0.5205818411628016]
[18.0 0.4925122353363278]
[19.0 0.46892135831389226]
[20.0 0.44197107931725554]
[21.0 0.4181125500953726]
[22.0 0.3957399664449426]
[23.0 0.3751527587957142]
[24.0 0.35717358210365546]
[25.0 0.33746154492081715]
[26.0 0.32099050569821586]
[27.0 0.3037475822746565]
[28.0 0.28894087699800863]
[29.0 0.2746346942714233]
[30.0 0.26113152049028443]
[31.0 0.24825271690610862]
[32.0 0.23606187030713954]
[33.0 0.22448908497934994]
[34.0 0.2134833658644446]
[35.0 0.20304412982334696]
[36.0 0.1931206135004082]
[37.0 0.18372187515130278]
[38.0 0.1747792424867569]
[39.0 0.1662

# 9. Plots
Finally, Zach wraps things up by plotting the equilibrium objects. It gets reduced to a two-dimensional plot for interpretability. Specifically, three lines are plotted - steady state output (y = 1.0), low output (y = 0.9), and high output (y = 1.1).
## 9.1 Pricing functions

In [ ]:
# Plot the solved policy and pricing functions
bN_plot = 500
b_plot_grid = bLopt:(bHopt-bLopt)/(bN_plot-1):bHopt

q_plot = zeros(bN_plot)
VR_plot = zeros(bN_plot)
VD_plot = zeros(bN_plot)
A_plot = zeros(bN_plot)
for i=1:bN_plot 
    q_plot[i] = q([1.0;b_plot_grid[i]])
    VR_plot[i] = vRepay([1.0;b_plot_grid[i]])
    A_plot[i] = Apol([1.0;b_plot_grid[i]])
    VD_plot[i] = vDefault(1.0)
end

b_grid = bL:.001:bH
# b_grid = bL:.03:bH
q_ss(b) = q([1.0, b])
q_l(b) = q([0.9, b])
q_h(b) = q([1.1, b])
plot(b_grid,q_ss.(b_grid),label="SS Output",xlabel="Debt Issuance",ylabel="Price")
plot!(b_grid,q_l.(b_grid),label="Low Output")
plot!(b_grid,q_h.(b_grid),label="High Output")

#savefig("pricing_functions_arellano_2008.pdf")

## 9.2 Bond issuance policy functions

In [1]:
a_b_alone_h(b) = Apol([1.1, b])
a_b_alone_l(b) = Apol([0.9, b])
plot(b_grid,a_b_alone_h.(b_grid),label="High Output",ylabel="Debt Issuance",xlabel="Debt Stock")
plot!(b_grid,a_b_alone_l.(b_grid),label="Low Output")

#savefig("policy_functions_arellano_2008.pdf")

LoadError: UndefVarError: `plot` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## 9.3 Value functions

In [2]:
y_grid = yL:.001:yH
btest = 0.2
V_y_alone(y) = vRepay([y; btest])
Vd_y_alone(y) = vDefault(y)

plot(y_grid,V_y_alone.(y_grid),xlabel="Output",label="Repayment Value")
plot!(y_grid,Vd_y_alone.(y_grid),label="Default Value")

#savefig("Value_functions_y_arellano_2008.pdf")

LoadError: UndefVarError: `yL` not defined in `Main`
Suggestion: check for spelling errors or missing imports.